In [4]:
# Imports 
import pandas as pd
import numpy as np

# Install latest version of pandas profiling
!pip install pandas-profiling==2.8.0
!pip install category_encoders
import pandas_profiling


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
# Import data
combined_csv = pd.read_csv('combined_csv.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,18,20,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
combined_csv.shape

(209445, 41)

In [7]:
# Datetime import for wrangle function
# from datetime import datetime

def wrangle(X):
  """ 
  Wrangles kickstarter data in preparation for binary classification
  prediction of success or failure.
  """
  
  # Copy to avoid SettingWithCopy error
  X = X.copy()

  # Reduce to only binary classification of "Successful" or "failed"
  X = X[(X["state"] == "successful") | (X["state"] == "failed")]

  # Convert target to binary variable
  X["success"] = X["state"].replace({"successful":1, "failed":0})

  # Create "Campaign Length" column & convert Unix to number of days
  X["campaign_length"] = (X["deadline"] - X["launched_at"]) / 86400

  # Drop any campaigns lasting longer than the allowed 60 days
  X = X[X["campaign_length"] < 61] 

  # Convert goal column to USD
  X["usd_goal"] = X["goal"] * X["fx_rate"]

  # Combine text columns for use in NLP modeling
  X["combined_text"] = X["name"] + " " + X["blurb"]

  # Pull category data from json category column
  categories = []
  for i in enumerate(X["category"]):
    df = pd.read_json(i[1])
    try:
      categories.append(df["parent_name"][0])
    except:
      categories.append(df["name"][0])
  X["category"] = categories

  return X

In [8]:
print(combined_csv.isna().sum())

backers_count                    0
blurb                            8
category                         0
converted_pledged_amount         0
country                          0
country_displayable_name         0
created_at                       0
creator                          0
currency                         0
currency_symbol                  0
currency_trailing_code           0
current_currency                 0
deadline                         0
disable_communication            0
friends                     209397
fx_rate                          0
goal                             0
id                               0
is_backing                  209397
is_starrable                     0
is_starred                  209397
last_update_published_at    209444
launched_at                      0
location                       215
name                             1
permissions                 209397
photo                            0
pledged                          0
profile             

In [9]:
combined_csv

backers_count  ...  usd_type
0                   4  ...  domestic
1                 404  ...  domestic
2                  20  ...  domestic
3                   6  ...  domestic
4                  39  ...  domestic
...               ...  ...       ...
209440            588  ...  domestic
209441              0  ...  domestic
209442            176  ...  domestic
209443             11  ...  domestic
209444             76  ...  domestic

[209445 rows x 41 columns]

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(combined_csv, 
                               stratify=combined_csv['state'])

In [11]:
print(train.isna().sum())
print(test.isna().sum())

backers_count                    0
blurb                            7
category                         0
converted_pledged_amount         0
country                          0
country_displayable_name         0
created_at                       0
creator                          0
currency                         0
currency_symbol                  0
currency_trailing_code           0
current_currency                 0
deadline                         0
disable_communication            0
friends                     157044
fx_rate                          0
goal                             0
id                               0
is_backing                  157044
is_starrable                     0
is_starred                  157044
last_update_published_at    157082
launched_at                      0
location                       161
name                             0
permissions                 157044
photo                            0
pledged                          0
profile             

In [12]:
# Wrangle data
train = wrangle(train)
test = wrangle(test)

In [13]:
test['combined_text'].isna()

95250     False
77777     False
161647    False
26996     False
5248      False
          ...  
205348    False
204990    False
48771     False
2519      False
35060     False
Name: combined_text, Length: 48677, dtype: bool

In [14]:
print(train.isna().sum())
print(test.isna().sum())

backers_count                    0
blurb                            2
category                         0
converted_pledged_amount         0
country                          0
country_displayable_name         0
created_at                       0
creator                          0
currency                         0
currency_symbol                  0
currency_trailing_code           0
current_currency                 0
deadline                         0
disable_communication            0
friends                     146046
fx_rate                          0
goal                             0
id                               0
is_backing                  146046
is_starrable                     0
is_starred                  146046
last_update_published_at    146046
launched_at                      0
location                       108
name                             0
permissions                 146046
photo                            0
pledged                          0
profile             

In [15]:
# Confirm correct categories added
train["state"].value_counts(normalize=True)

successful    0.622537
failed        0.377463
Name: state, dtype: float64

In [16]:
train.shape

(146046, 45)

In [17]:
# Define X matrix and y target to prepare for baseline model
features = ["combined_text", "campaign_length", "category", "usd_goal"]
target = ["success"]

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [18]:
X_train

combined_text  ...     usd_goal
165860  2020 NightScaper Conference Astro-landscape co...  ...  10000.00000
137745  Erweiterung meiner Hobbybrauerei Hallo,\r\nMei...  ...    562.47525
45779   Something In The Way Project Help Sam print an...  ...   7500.00000
131283  Fragments of Memory: a multimedia project in a...  ...   1882.64826
38159   Sophia Angeles CD Recording Project I'm record...  ...   2500.00000
...                                                   ...  ...          ...
30599   Fantasy Novel: Age of Legend by Michael J. Sul...  ...  22240.00000
10660   Bang! Bang! BOOM! Vol 1 A jazz era romantic ad...  ...   5425.00000
129219  NIGHTBLIND: Beneath the Alien Plains - a sci-f...  ...   1200.00000
161222  The Fucking Calendar This is just a calendar f...  ...  11249.50500
57444   Recuerdo Celestial Una novela que narra cómo u...  ...   2018.72475

[146046 rows x 4 columns]

In [19]:
# Check for NaNs
print(X_train.isna().sum())
print(X_test.isna().sum())
print(y_train.isna().sum())
print(y_test.isna().sum())

combined_text      2
campaign_length    0
category           0
usd_goal           0
dtype: int64
combined_text      0
campaign_length    0
category           0
usd_goal           0
dtype: int64
success    0
dtype: int64
success    0
dtype: int64


In [21]:
y_train.tail

<bound method NDFrame.tail of         success
165860        1
137745        0
45779         1
131283        1
38159         1
...         ...
30599         1
10660         1
129219        1
161222        0
57444         1

[146046 rows x 1 columns]>

In [22]:
# X_test = X_test.dropna()
# X_train = X_train.dropna()
# y_train = y_train.drop([57444, 161222], axis=0)
# y_test = y_test.drop([41152], axis=0)

In [24]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(146044, 4)
(48677, 4)
(146044, 1)
(48677, 1)


In [25]:
# Categorical encode category column
import category_encoders as ce

encoder = ce.OrdinalEncoder()
X_train["category"] = encoder.fit_transform(X_train["category"])
X_test["category"] = encoder.transform(X_test["category"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
X_train.head()

combined_text  ...     usd_goal
165860  2020 NightScaper Conference Astro-landscape co...  ...  10000.00000
137745  Erweiterung meiner Hobbybrauerei Hallo,\r\nMei...  ...    562.47525
45779   Something In The Way Project Help Sam print an...  ...   7500.00000
131283  Fragments of Memory: a multimedia project in a...  ...   1882.64826
38159   Sophia Angeles CD Recording Project I'm record...  ...   2500.00000

[5 rows x 4 columns]

In [27]:
# Imports for word vectorizations
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [28]:
# Define Pipeline
pipe = Pipeline([
                 ('vect', vect),
                 ('clf', rfc)
                 ])

In [29]:
y_test.tail()
# X_train.shape

success
205348        1
204990        1
48771         1
2519          0
35060         1

In [30]:
X_test.shape

(48677, 4)

In [31]:
# Define parameters for search
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (1000,3000),
    'clf__n_estimators': (5,10),
    'clf__max_depth': (15, 20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=True)
grid_search.fit(X_train["combined_text"], y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 26.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [32]:
# Look at best score
grid_search.best_score_

0.6328228368133815

In [33]:
# Create new columns in df with prediction & probability by row
X_train["nlp_pred"] = grid_search.predict(X_train["combined_text"])
X_train["nlp_proba"] = grid_search.predict_proba(X_train["combined_text"])[:,1]

X_test["nlp_pred"] = grid_search.predict(X_test["combined_text"])
X_test["nlp_proba"] = grid_search.predict_proba(X_test["combined_text"])[:,1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [34]:
# Double check new DF
X_train.head()

combined_text  ...  nlp_proba
165860  2020 NightScaper Conference Astro-landscape co...  ...   0.595033
137745  Erweiterung meiner Hobbybrauerei Hallo,\r\nMei...  ...   0.595033
45779   Something In The Way Project Help Sam print an...  ...   0.583806
131283  Fragments of Memory: a multimedia project in a...  ...   0.583806
38159   Sophia Angeles CD Recording Project I'm record...  ...   0.608894

[5 rows x 6 columns]

In [35]:
# Redefine X matrix for final model
final_features = ["campaign_length", "category", "usd_goal", "nlp_proba"]
final_features2 = ["campaign_length", "category", "usd_goal"]

X_train_final = X_train[final_features]
X_test_final = X_test[final_features]

In [36]:
# Create final model
model = RandomForestClassifier()

parameters = {
    'n_estimators': (5, 10, 50),
    'max_depth': (10, 15, 20)
}

grid_search_final = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=True)
grid_search_final.fit(X_train_final, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.8min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [37]:
grid_search_final.best_score_

0.6883877141288128

In [38]:
grid_search_final.best_params_

{'max_depth': 15, 'n_estimators': 50}

In [ ]:
grid_search.score(X_test_final, y_test)

In [39]:
X_test_final.shape

(48677, 4)

In [40]:
y_test.shape

(48677, 1)

In [41]:
from tensorflow import keras
import tensorflow as tf

In [ ]:
X_train['campaign_length'] = X_train['campaign_length'].astype(int)
X_train['category'] = X_train['category'].astype(int)
X_train['usd_goal'] = X_train['usd_goal'].astype(int)

In [ ]:
meta_input = X_train[final_features2]

In [ ]:
nlp_input = X_train['nlp_pred']

In [42]:
X_train_final

campaign_length  category     usd_goal  nlp_proba
165860        30.000000         1  10000.00000   0.595033
137745        60.000000         2    562.47525   0.595033
45779         30.000000         1   7500.00000   0.583806
131283        31.138484         3   1882.64826   0.583806
38159         40.000000         4   2500.00000   0.608894
...                 ...       ...          ...        ...
30599         20.250394        12  22240.00000   0.885111
10660         30.462419        10   5425.00000   0.595033
129219        30.000000         7   1200.00000   0.600000
161222        35.000000        14  11249.50500   0.607479
57444         28.000000        12   2018.72475   0.595033

[146044 rows x 4 columns]

In [43]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_final = scaler.fit_transform(X_train_final)
X_test_final = scaler.transform(X_test_final)

In [44]:
from sklearn.linear_model import LogisticRegression

In [45]:
model = LogisticRegression()

# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

grid_search_final = GridSearchCV(model, hyperparameters, cv=5, n_jobs=-1, verbose=True)
grid_search_final.fit(X_train_final, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    9.8s finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             s

In [46]:
grid_search_final.best_params_

{'C': 7.742636826811269, 'penalty': 'l2'}

In [47]:
grid_search_final.best_score_

0.6600129937945028

In [48]:
from sklearn.svm import SVC 

In [49]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']
#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto")
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto")
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto")
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto")

In [ ]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma':
              [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

In [ ]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train_final, y_train)

In [ ]:
print(grid.best_estimator_)

In [ ]:
grid_predictions = grid.predict(X_test_final)

In [ ]:
grid_search.score(y_test, grid_predictions)

In [112]:
inputs = X_train_final.shape[1]
epochs = 1000
batch_size = 1000


# Create Model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation='tanh', input_shape=(inputs,)))
model.add(tf.keras.layers.Dense(128, activation='tanh'))
model.add(tf.keras.layers.Dense(64, activation='tanh'))
model.add(tf.keras.layers.Dense(32, activation='tanh'))
# model.add(tf.keras.layers.Dense(32, activation='sigmoid'))
model.add(tf.keras.layers.Dense(16, activation='tanh'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit Model
model.fit(X_train_final, y_train, 
          validation_data=(X_test_final, y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Epoch 1/1000
147/147 [==============================] - 1s 4ms/step - loss: 0.6539 - accuracy: 0.6287 - val_loss: 0.6572 - val_accuracy: 0.6227
Epoch 2/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6526 - accuracy: 0.6307 - val_loss: 0.6585 - val_accuracy: 0.6219
Epoch 3/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6525 - accuracy: 0.6305 - val_loss: 0.6625 - val_accuracy: 0.6208
Epoch 4/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6525 - accuracy: 0.6305 - val_loss: 0.6563 - val_accuracy: 0.6209
Epoch 5/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6522 - accuracy: 0.6307 - val_loss: 0.6593 - val_accuracy: 0.6218
Epoch 6/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6524 - accuracy: 0.6307 - val_loss: 0.6570 - val_accuracy: 0.6228
Epoch 7/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6522 - accuracy: 0.6309 - val_loss: 0.6584 - val_accuracy:

In [110]:
# my_init = keras.initializers.glorot_uniform(seed=1)
# model = keras.models.Sequential()
# model.add(keras.layers.Dense(units=8, input_dim=4,
#   activation='tanh', kernel_initializer=my_init)) 
# model.add(keras.layers.Dense(units=8, activation='tanh',
#   kernel_initializer=my_init)) 
# model.add(keras.layers.Dense(units=1, activation='sigmoid',
#   kernel_initializer=my_init))
# simple_sgd = keras.optimizers.SGD(lr=0.1)  
# model.compile(loss='binary_crossentropy',
#   optimizer=simple_sgd, metrics=['accuracy'])

In [111]:
model.fit(X_train_final, y_train, 
          validation_data=(X_test_final, y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Epoch 1/1000
147/147 [==============================] - 1s 4ms/step - loss: 0.6624 - accuracy: 0.6143 - val_loss: 0.6556 - val_accuracy: 0.6205
Epoch 2/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6531 - accuracy: 0.6304 - val_loss: 0.6558 - val_accuracy: 0.6215
Epoch 3/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6529 - accuracy: 0.6300 - val_loss: 0.6564 - val_accuracy: 0.6208
Epoch 4/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6529 - accuracy: 0.6302 - val_loss: 0.6567 - val_accuracy: 0.6211
Epoch 5/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6528 - accuracy: 0.6301 - val_loss: 0.6569 - val_accuracy: 0.6209
Epoch 6/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6528 - accuracy: 0.6301 - val_loss: 0.6566 - val_accuracy: 0.6210
Epoch 7/1000
147/147 [==============================] - 0s 3ms/step - loss: 0.6527 - accuracy: 0.6300 - val_loss: 0.6562 - val_accuracy:

KeyboardInterrupt: ignored